# Query analysis

## Problem
In any question answering application we need to search for, or retrieve, information based on a user question. In the simplest case, we can search on the user input directly. This approach has a few common failure modes:

* The data has multiple attributes that a user input could be referring to,
* The user input contains multiple distinct questions in it,
* Search quality is sensitive to phrasing.

To handle these, we can do **query analysis** to translate the raw user question into a query or queries optimized for our index. 

**NOTE**: This guide assumes familiarity with the basic building blocks of a simple RAG application outlined in the [Quickstart](/docs/use_cases/question_answering/quickstart).

## Solution

Query analysis is the process of transforming a user input into a query optimized for your index(es). This can involve any of the folowing steps:

* **Query decomposition**: If a user input contains multiple distinct questions, we can decompose the input into separate queries that will each be executed independently.
* **Query expansion**: If an index is sensitive to query phrasing, we can multiple paraphrased versions of the user question to increase our chances of retrieving a relevant result.
* **Query structuring**: If our documents have multiple searchable/filterable attributes, we can infer from any raw user question which specific attributes should be searched/filtered over. For example, when a user input specific something about video publication date, that should become a filter on the `publish_date` attribute of each document.
* **Routing**: If we have multiple indexes and only a subset are useful for any given user input, we can route the input to only retrieve results from the relevant ones.

Which of these steps are needed depends on the indexes we've created, which ultimately depends on the use case. The types of indexes we create and the query analysis steps we'll need might look very different when retrieving from a blog versus retrieving from a codebase. We'll cover all of the techniques at a high level in this guide, including when they're most useful and how you can optimize them.

To illustrate, let's build a Q&A bot over the LangChain YouTube videos.

## Setup
#### Install dependencies

In [ ]:
# %pip install -qU langchain langchain-community langchain-openai youtube-transcript-api pytube elasticsearch

#### Set environment variables

We'll use OpenAI in this example:

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

# Optional, uncomment to trace runs with LangSmith. Sign up here: https://smith.langchain.com.
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

### Set up integrations

We'll use Elasticsearch for our vectorstore. We can run an Elasticsearch instance locally with Docker:

```bash
docker run -p 9200:9200 -e "discovery.type=single-node" -e "xpack.security.enabled=false" -e "xpack.security.http.ssl.enabled=false" docker.elastic.co/elasticsearch/elasticsearch:8.9.0
```

### Load documents

We can use the `YouTubeLoader` to load transcripts of a few LangChain videos:

In [2]:
from langchain_community.document_loaders import YoutubeLoader

urls = [
    "https://www.youtube.com/watch?v=pbAd8O1Lvm4",
    "https://www.youtube.com/watch?v=ylrew7qb8sQ",
    "https://www.youtube.com/watch?v=uRya4zRrRx4",
    "https://www.youtube.com/watch?v=hvAPnpSfSGo",
    "https://www.youtube.com/watch?v=ZcEMLz27sL4",
    "https://www.youtube.com/watch?v=3wAON0Lqviw",
    "https://www.youtube.com/watch?v=jx7xuHlfsEQ",
    "https://www.youtube.com/watch?v=xn1jEjRyJ2U",
    "https://www.youtube.com/watch?v=SaDzIVkYqyY",
    "https://www.youtube.com/watch?v=gqhlqdawHT4",
    "https://www.youtube.com/watch?v=Ce03oEotdPs",
    "https://www.youtube.com/watch?v=rZus0JtRqXE",
    "https://www.youtube.com/watch?v=HAn9vnJy6S4",
    "https://www.youtube.com/watch?v=dA1cHGACXCo",
    "https://www.youtube.com/watch?v=ZcEMLz27sL4",
    "https://www.youtube.com/watch?v=hvAPnpSfSGo",
    "https://www.youtube.com/watch?v=EhlPDL4QrWY",
    "https://www.youtube.com/watch?v=mmBo8nlu2j0",
    "https://www.youtube.com/watch?v=rQdibOsL1ps",
    "https://www.youtube.com/watch?v=28lC4fqukoc",
    "https://www.youtube.com/watch?v=es-9MgxB-uc",
    "https://www.youtube.com/watch?v=wLRHwKuKvOE",
    "https://www.youtube.com/watch?v=ObIltMaRJvY",
    "https://www.youtube.com/watch?v=DjuXACWYkkU",
    "https://www.youtube.com/watch?v=o7C9ld6Ln-M",
]
docs = []
for url in urls:
    docs.extend(YoutubeLoader.from_youtube_url(url, add_video_info=True).load())

Here are the titles of the videos we've loaded:

In [3]:
[doc.metadata["title"] for doc in docs]

['Self-reflective RAG with LangGraph: Self-RAG and CRAG',
 'WebVoyager',
 'LangGraph: Planning Agents',
 'LangGraph: Multi-Agent Workflows',
 'Streaming Events: Introducing a new `stream_events` method',
 'LangSmith: In-Depth Platform Overview',
 'LangSmith in 10 Minutes',
 'RAG from scratch: Part 8 (Query Translation -- Step Back)',
 'RAG from scratch: Part 9 (Query Translation -- HyDE)',
 'RAG from scratch: Part 7 (Query Translation -- Decomposition - v1)',
 'LangChain Agents with Open Source Models!',
 'Gemini + Google Retrieval Agent from a LangChain Template',
 'OpenGPTs',
 'Building a web RAG chatbot: using LangChain, Exa (prev. Metaphor), LangSmith, and Hosted Langserve',
 'Streaming Events: Introducing a new `stream_events` method',
 'LangGraph: Multi-Agent Workflows',
 'Build and Deploy a RAG app with Pinecone Serverless',
 'Auto-Prompt Builder (with Hosted LangServe)',
 'Build a Full Stack RAG App With TypeScript',
 'Getting Started with Multi-Modal LLMs',
 'SQL Research Assi

Here's the metadata associated with each video. We can see that each document also has a title, view count, publication date, and length:

In [5]:
docs[0].metadata

{'source': 'pbAd8O1Lvm4',
 'title': 'Self-reflective RAG with LangGraph: Self-RAG and CRAG',
 'description': 'Unknown',
 'view_count': 7946,
 'thumbnail_url': 'https://i.ytimg.com/vi/pbAd8O1Lvm4/hq720.jpg',
 'publish_date': '2024-02-07 00:00:00',
 'length': 1058,
 'author': 'LangChain'}

And here's a sample from a document's contents:

In [4]:
docs[0].page_content[:500]

"hi this is Lance from Lang chain I'm going to be talking about using Lang graph to build a diverse and sophisticated rag flows so just to set the stage the basic rag flow you can see here starts with a question retrieval of relevant documents from an index which are passed into the context window of an llm for generation of an answer grounded in the ret documents so that's kind of the basic outline and we can see it's like a very linear path um in practice though you often encounter a few differ"

### Indexing documents

Whenever we perform retrieval we need to create an index of documents that we can query. We'll use a vector store to index our documents, and we'll chunk them first to make our retrievals more concise and precise:

In [183]:
import datetime

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma, ElasticsearchStore
from langchain_openai import OpenAIEmbeddings

# clean up metadata
for doc in docs:
    doc.metadata["publish_date"] = datetime.datetime.strptime(
        doc.metadata["publish_date"], "%Y-%m-%d %H:%M:%S"
    ).strftime("%Y-%m-%d")


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000, chunk_overlap=500, add_start_index=True
)
chunked_docs = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = ElasticsearchStore.from_documents(
    chunked_docs,
    embeddings,
    index_name="langchain_youtube_2",
    es_url="http://localhost:9200",
)

## Retrieval without query analysis

We can perform similarity search on a user question directly to find chunks relevant to the question:

In [7]:
search_results = vectorstore.similarity_search("how do I build a RAG agent")
print(search_results[0].metadata["title"])
print(search_results[0].page_content[:500])

OpenGPTs
it decides to use a tool and so importantly it lets us know that it's deciding to use a tool and then it also lets us know what the result of the tool is and then it starts streaming back the response so this is streaming not just tokens but also these intermediate steps which provide really good visibility into what is going on we can see here we can see the response that we got back from tavil um and then we can see um the response from the AI and so there's lots of dad jokes in here this is u


This works pretty well! Our first result is quite relevant to the question.

Now what if we remembered that there was a video series titled "RAG from scratch" and wanted to find that specifically?

In [8]:
search_results = vectorstore.similarity_search(
    "rag from scratch",
)
print(search_results[0].metadata["title"])
print(search_results[0].page_content[:500])

Self-reflective RAG with LangGraph: Self-RAG and CRAG
hi this is Lance from Lang chain I'm going to be talking about using Lang graph to build a diverse and sophisticated rag flows so just to set the stage the basic rag flow you can see here starts with a question retrieval of relevant documents from an index which are passed into the context window of an llm for generation of an answer grounded in the ret documents so that's kind of the basic outline and we can see it's like a very linear path um in practice though you often encounter a few differ


In [9]:
[res.metadata["title"] for res in search_results]

['Self-reflective RAG with LangGraph: Self-RAG and CRAG',
 'Build and Deploy a RAG app with Pinecone Serverless',
 'OpenGPTs',
 'LangServe and LangChain Templates Webinar']

Since we're only searching over the transcriptions, and not over titles, our search misses all of the relevant documents. 

What if we wanted to search for results from a specific time period?

In [10]:
search_results = vectorstore.similarity_search("videos on RAG published in 2023")
print(search_results[0].metadata["title"])
print(search_results[0].metadata["publish_date"])
print(search_results[0].page_content[:500])

OpenGPTs
2024-01-31 00:00:00
it decides to use a tool and so importantly it lets us know that it's deciding to use a tool and then it also lets us know what the result of the tool is and then it starts streaming back the response so this is streaming not just tokens but also these intermediate steps which provide really good visibility into what is going on we can see here we can see the response that we got back from tavil um and then we can see um the response from the AI and so there's lots of dad jokes in here this is u


Our first result is from 2024, and not very relevant to the input. Since we're just searching against document contents, there's no way for the results to be filtered on any document attributes.

What if we wanted to know about deploying a LangChain chain as a REST API?

In [11]:
search_results = vectorstore.similarity_search("chain as rest api")
print(search_results[0].metadata["title"])
print(search_results[0].page_content[1500:2000])

Building a web RAG chatbot: using LangChain, Exa (prev. Metaphor), LangSmith, and Hosted Langserve
 further and see um that the inputs to each of those kind of Lambda steps is going to be one of those documents and then we're outputting um like the highlights DL segment and then formatting that with our prompt template uh if you recall from when we were constructing it um so now we have kind of our uh fully constructed chain uh for our uh search enable chatbot with XF um and now let's convert that to Lang serve um so to do that we'll go back to vs code um and here we're going to um start with


This brings up LangServe, the package for deploying chains as REST API's, as desired.

But what if we added that we wanted a chain that made use of multi-modal models?

In [86]:
search_results = vectorstore.similarity_search(
    "how to use multi-modal models in a chain and turn chain into a rest api"
)
print(search_results[0].metadata["title"])
print(search_results[0].page_content[:500])

Streaming Events: Introducing a new `stream_events` method
streaming is uh an incredibly important ux consideration for building L Ms in a few ways first of all even if you're just working with a single llm call it can often take a while and you might want to stream individual tokens to the user so they can see what's happening as the llm responds second of all a lot of the things that we build in the laying chain are more complicated chains or agents and so being able to stream the intermediate steps what tool are being called what the input to those t


Our first result ends up not being about LangServe or multi-modal models. In reality "chains as rest API" and "using multi-modal models" are two fairly distinct questions that should be queried for separately.

## Query analysis

To handle these failure modes we can perform **query analysis**. Specifically, we can perform:

* **Query structuring**: If our documents have multiple searchable/filterable attributes, we can infer from any raw user question which specific attributes should be searched/filtered over. For example, when a user input specific something about video publication date, that should become a filter on the `publish_date` attribute of each document.
* **Query decomposition**: If a user input contains multiple distinct questions, we can decompose the input into separate queries.
* **Query expansion**: If an index is sensitive to query phrasing, we can multiple paraphrased versions of the user question to increase our chances of retrieving a relevant result.

To do this we'll define a **query schema** and use a function-calling model to convert a user question into a structured query or queries. The structured nature of the query schema allows us to do query structuring and routing, and the fact that we can extract multiple of these 
allows us to do decomposition and expansion.

### Query schema
In this case we'll have explicit min and max attributes for view count, publication date, and video length so that those can be filtered on. And we'll add separate attributes for searches against the transcript contents versus the video title. We'll also add some sorting attributes that we'll touch on later.

In [67]:
import datetime
from typing import Literal, Optional, Tuple

from langchain_core.pydantic_v1 import BaseModel, Field


class TutorialSearch(BaseModel):
    """Search over a database of tutorial videos about a software library."""

    content_search: str = Field(
        ...,
        description="Similarity search query applied to video transcripts.",
    )
    title_search: str = Field(
        ...,
        description=(
            "Alternate version of the content search query to apply to video titles. "
            "Should be succinct and only include key words that could be in a video "
            "title."
        ),
    )
    min_view_count: Optional[int] = Field(
        None, description="Minimum view count filter, inclusive."
    )
    max_view_count: Optional[int] = Field(
        None, description="Maximum view count filter, exclusive."
    )
    earliest_publish_date: Optional[datetime.date] = Field(
        None, description="Earliest publish date filter, inclusive."
    )
    latest_publish_date: Optional[datetime.date] = Field(
        None, description="Latest publish date filter, exclusive."
    )
    min_length_sec: Optional[int] = Field(
        None, description="Minimum video length in seconds, inclusive."
    )
    max_length_sec: Optional[int] = Field(
        None, description="Maximum video length in seconds, exclusive."
    )
    sort_by: Literal[
        "relevance",
        "view_count",
        "publish_date",
        "length",
    ] = Field("relevance", description="Attribute to sort by.")
    sort_order: Literal["ascending", "descending"] = Field(
        "descending", description="Whether to sort in ascending or descending order."
    )
    relevance_rank: int = Field(
        ...,
        description=(
            "The index of this search query when all generated queries are sorted by "
            "the expected relevance of their results to the original user question. "
            "Each query must have a distinct index. A lower rank indicates higher "
            "relevance to the user question."
        ),
    )

    def pretty_print(self) -> None:
        for field in self.__fields__:
            if getattr(self, field) is not None and getattr(self, field) != getattr(
                self.__fields__[field], "default", None
            ):
                print(f"{field}: {getattr(self, field)}")

### Query generation

To convert user questions to structured queries we'll make use of OpenAI's function-calling API. Since the latest OpenAI models can return multiple function invocations each turn, this approach automatically supports query expansion and decomposition.

In [72]:
from langchain.output_parsers import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

system = """You are an expert at converting user questions into database queries. \
You have access to a database of tutorial videos about a software library for building LLM-powered applications. \
Given a question, return a list of database queries optimized to retrieve the most relevant results.

Perform query expansion. If there are multiple common ways of phrasing a user question \
or common synonyms for key words in the question, make sure to return multiple versions \
of the query with the different phrasings.

Perform query decomposition. If the user input contains a multi-part question, make \
sure to return a separate query for each distinct sub-question.

If there are acronyms or words you are not familiar with, do not try to rephrase them."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        MessagesPlaceholder("examples", optional=True),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
llm_with_tools = llm.bind_tools([TutorialSearch])
query_analyzer = (
    {"question": RunnablePassthrough()}
    | prompt
    | llm_with_tools
    | PydanticToolsParser(tools=[TutorialSearch])
)

Let's see what queries our analyzer generates for the questions we searched earlier:

In [73]:
for query in query_analyzer.invoke("rag from scratch"):
    query.pretty_print()
    print()

content_search: RAG from scratch
title_search: RAG
relevance_rank: 1

content_search: Reactive Agile Governance from scratch
title_search: Reactive Agile Governance
relevance_rank: 2

content_search: How to build RAG applications from the beginning
title_search: RAG applications
relevance_rank: 3



In [74]:
for query in query_analyzer.invoke("videos on RAG published in 2023"):
    query.pretty_print()
    print()

content_search: RAG
title_search: RAG
earliest_publish_date: 2023-01-01
latest_publish_date: 2024-01-01
relevance_rank: 1



In [85]:
for query in query_analyzer.invoke(
    "how to use multi-modal models in a chain and turn chain into a rest api"
):
    query.pretty_print()
    print()

content_search: multi-modal models chain
title_search: multi-modal models chain
relevance_rank: 1

content_search: chain into REST API
title_search: chain REST API
relevance_rank: 2



### Improvements: Adding examples to the prompt

To tune our results we can add some examples of inputs questions and gold standard output queries to our prompt. We'll focus on examples that show how to route and expand queries, to either be against titles or content, how to structure them with filters, and how to decompose them:

In [76]:
examples = []

In [77]:
question = "What is Web Voyager? How about Gemini?"
queries = [
    TutorialSearch(
        content_search="what is Web Voyager",
        title_search="Web Voyager",
        relevance_rank=1,
    ),
    TutorialSearch(
        content_search="What is Gemini", title_search="Gemini", relevance_rank=2
    ),
]
examples.append({"input": question, "tool_calls": queries})

In [78]:
question = "Have they released any chat langchain updates since 2024?"
queries = [
    TutorialSearch(
        title_search="chat langchain",
        content_search="chat langchain",
        earliest_publish_date=datetime.date(2024, 1, 1),
        relevance_rank=1,
    ),
]
examples.append({"input": question, "tool_calls": queries})

In [79]:
question = "How to build multi-agent system and stream intermediate steps from it"
queries = [
    TutorialSearch(
        content_search="How to build multi-agent system",
        title_search="multi-agent system",
        relevance_rank=1,
    ),
    TutorialSearch(
        content_search="how to stream intermediate steps from multi-agent system",
        title_search="stream intermediate steps multi-agent system",
        relevance_rank=2,
    ),
    TutorialSearch(
        content_search="how to stream intermediate steps",
        title_search="stream intermediate steps",
        relevance_rank=3,
    ),
]
examples.append({"input": question, "tool_calls": queries})

Now we need to update our prompt template and chain so that the examples are included in each prompt. Since we're working with OpenAI function-calling, we'll need to do a bit of extra structuring to send example inputs and outputs to the model. We'll create a `tool_example_to_messages` helper function to handle this for us:

In [42]:
import uuid
from typing import Dict, List

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
)


def tool_example_to_messages(example: Dict) -> List[BaseMessage]:
    messages: List[BaseMessage] = [HumanMessage(content=example["input"])]
    openai_tool_calls = []
    for tool_call in example["tool_calls"]:
        openai_tool_calls.append(
            {
                "id": str(uuid.uuid4()),
                "type": "function",
                "function": {
                    "name": tool_call.__class__.__name__,
                    "arguments": tool_call.json(),
                },
            }
        )
    messages.append(
        AIMessage(content="", additional_kwargs={"tool_calls": openai_tool_calls})
    )
    tool_outputs = example.get("tool_outputs") or [
        "This is an example of a correct usage of this tool. Well done. Make sure to continue using the tool this way."
    ] * len(openai_tool_calls)
    for output, tool_call in zip(tool_outputs, openai_tool_calls):
        messages.append(ToolMessage(content=output, tool_call_id=tool_call["id"]))
    return messages


example_msgs = [msg for ex in examples for msg in tool_example_to_messages(ex)]
query_analyzer_with_examples = (
    {"question": RunnablePassthrough()}
    | prompt.partial(examples=example_msgs)
    | llm_with_tools
    | PydanticToolsParser(tools=[TutorialSearch])
)

In [81]:
for query in query_analyzer_with_examples.invoke("rag from scratch"):
    query.pretty_print()
    print()

content_search: rag from scratch
title_search: rag from scratch
relevance_rank: 1



In [84]:
for query in query_analyzer_with_examples.invoke(
    "how to use multi-modal models in a chain and turn chain into a rest api"
):
    query.pretty_print()
    print()

content_search: how to use multi-modal models in a chain
title_search: multi-modal models chain
relevance_rank: 1

content_search: how to turn chain into a REST API
title_search: chain REST API
relevance_rank: 2



In [83]:
for query in query_analyzer_with_examples.invoke(
    "How to do extraction with agent? How to build agent with anthropic"
):
    query.pretty_print()
    print()

content_search: How to do extraction with agent
title_search: extraction with agent
relevance_rank: 1

content_search: How to build agent with anthropic
title_search: build agent anthropic
relevance_rank: 2



## Retrieval with query analysis

Our query analysis looks pretty good; now let's try using our generated queries to actually perform retrieval. We'll define a custom retrieval lambda that takes our output queries and correctly applies them to our indexes. 

Before we do that, we'll also need to create a separate index since we created an additional search field, `title_search`, for searching against video titles specifically.

In [184]:
from copy import deepcopy

from langchain.storage import InMemoryStore
from langchain_core.documents import Document

docstore = InMemoryStore()
docstore.mset([(doc.metadata["source"], doc) for doc in docs])

title_docs = []
for doc in docs:
    metadata = deepcopy(doc.metadata)
    title_docs.append(Document(metadata.pop("title"), metadata=metadata))
title_vectorstore = ElasticsearchStore.from_documents(
    title_docs,
    embeddings,
    index_name="langchain_youtube_titles_3",
    es_url="http://localhost:9200",
)

In [201]:
from typing import List

from langchain.chains.query_constructor.ir import (
    Comparator,
    Comparison,
    Operation,
    Operator,
    StructuredQuery,
)
from langchain.retrievers.self_query.elasticsearch import ElasticsearchTranslator
from langchain_core.documents import Document


def query_to_filter(query: TutorialSearch) -> dict:
    comparisons = []
    if query.min_view_count is not None:
        comparisons.append(
            Comparison(
                comparator=Comparator.GTE,
                attribute="view_count",
                value=query.min_view_count,
            )
        )
    if query.max_view_count is not None:
        comparisons.append(
            Comparison(
                comparator=Comparator.LT,
                attribute="view_count",
                value=query.max_view_count,
            )
        )
    if query.min_length_sec is not None:
        comparisons.append(
            Comparison(
                comparator=Comparator.GTE,
                attribute="length",
                value=query.min_length_sec,
            )
        )
    if query.max_length_sec is not None:
        comparisons.append(
            Comparison(
                comparator=Comparator.LT, attribute="length", value=query.max_length_sec
            )
        )
    if query.earliest_publish_date is not None:
        comparisons.append(
            Comparison(
                comparator=Comparator.GTE,
                attribute="publish_date",
                value={"type": "date", "date": query.earliest_publish_date},
            )
        )
    if query.latest_publish_date is not None:
        comparisons.append(
            Comparison(
                comparator=Comparator.LT,
                attribute="publish_date",
                value={"type": "date", "date": query.latest_publish_date},
            )
        )
    if comparisons:
        filter = Operation(operator=Operator.AND, arguments=comparisons)
        return ElasticsearchTranslator().visit_operation(filter)
    else:
        return {}


def content_search(input: dict) -> List[Document]:
    return vectorstore.similarity_search_with_score(
        input["query"].content_search, filter=input["filter"]
    )


def title_search(input: dict) -> List[Document]:
    title_docs_scores = title_vectorstore.similarity_search_with_score(
        input["query"].title_search, filter=input["filter"]
    )
    docs = docstore.mget(
        [title_doc.metadata["source"] for title_doc, _ in title_docs_scores]
    )
    scores = [scores for _, scores in title_docs_scores]
    return list(zip(docs, scores))

In [202]:
def queries_and_filters(queries: List[TutorialSearch]) -> List[Dict]:
    return [{"query": q, "filter": query_to_filter(q)} for q in queries]


search = RunnablePassthrough.assign(
    content_docs=content_search, title_docs=title_search
)
retrieval = query_analyzer_with_examples | queries_and_filters | search.map()

In [135]:
results = retrieval.invoke("rag from scratch")

In [139]:
[doc.metadata["title"] for result in results for doc, _ in result["content_docs"]]

['Self-reflective RAG with LangGraph: Self-RAG and CRAG',
 'Build and Deploy a RAG app with Pinecone Serverless',
 'OpenGPTs',
 'LangServe and LangChain Templates Webinar']

In [140]:
[doc.metadata["title"] for result in results for doc, _ in result["title_docs"]]

['Self-reflective RAG with LangGraph: Self-RAG and CRAG',
 'RAG from scratch: Part 8 (Query Translation -- Step Back)',
 'Build a Full Stack RAG App With TypeScript',
 'RAG from scratch: Part 7 (Query Translation -- Decomposition - v1)']

In [141]:
results = retrieval.invoke(
    "how to use multi-modal models in a chain and turn chain into a rest api"
)

In [142]:
[doc.metadata["title"] for result in results for doc, _ in result["content_docs"]]

['Streaming Events: Introducing a new `stream_events` method',
 'Streaming Events: Introducing a new `stream_events` method',
 'Getting Started with Multi-Modal LLMs',
 'LangChain Agents with Open Source Models!',
 'Build and Deploy a RAG app with Pinecone Serverless',
 'Building a Research Assistant from Scratch',
 'Build and Deploy a RAG app with Pinecone Serverless',
 'Building a web RAG chatbot: using LangChain, Exa (prev. Metaphor), LangSmith, and Hosted Langserve']

In [143]:
[doc.metadata["title"] for result in results for doc, _ in result["title_docs"]]

['LangChain Agents with Open Source Models!',
 'Getting Started with Multi-Modal LLMs',
 'Building a web RAG chatbot: using LangChain, Exa (prev. Metaphor), LangSmith, and Hosted Langserve',
 'LangGraph: Multi-Agent Workflows',
 'Build and Deploy a RAG app with Pinecone Serverless',
 'Benchmarking RAG over LangChain Docs',
 'Build a Full Stack RAG App With TypeScript',
 'Building a web RAG chatbot: using LangChain, Exa (prev. Metaphor), LangSmith, and Hosted Langserve']

In [189]:
results = retrieval.invoke("RAG tutorial published in 2023")

In [190]:
[
    (doc.metadata["title"], doc.metadata["publish_date"])
    for result in results
    for doc, _ in result["content_docs"]
]

[('Getting Started with Multi-Modal LLMs', '2023-12-20'),
 ('LangServe and LangChain Templates Webinar', '2023-11-02'),
 ('Getting Started with Multi-Modal LLMs', '2023-12-20'),
 ('SQL Research Assistant', '2023-12-19')]

In [193]:
results = retrieval.invoke("short tutorial (less than 15 min) on agents")

In [194]:
[
    (doc.metadata["title"], doc.metadata["length"])
    for result in results
    for doc, _ in result["content_docs"]
]

[('Gemini + Google Retrieval Agent from a LangChain Template', 776),
 ('Gemini + Google Retrieval Agent from a LangChain Template', 776),
 ('Gemini + Google Retrieval Agent from a LangChain Template', 776),
 ('LangSmith in 10 Minutes', 561)]

### Generation

Most of the time we're doing retrieval so that we can generate an informed response. To do this we'll need to consolidate our current retrieval results. Currently we return a search result for each constructed query, and that result contains both document chunks that match the query and full documents whose titles match the query. There's a few possible ways we could combine these results:

* Dump all docs in the prompt. This might work fine for models with large context windows, but could cause issues with smaller windows since the results from the title search can be quite long.
* Compress title docs. If our title docs are too large to pass to the model, we can do some post-processing to chunk them and discard all but the most relevant chunks.
* Use title docs to rerank chunked docs. We could use the returned titles to rerank the chunked docs by taking into account how relevant their titles are. We might fetch 20 chunks and 20 titles, rerank the chunks based on their content + title relevance, and then include the top 10 reranked chunks in the prompt.

To keep things simple here we'll go with the first approach, which is including all results in the model prompt. To account for the fact that title docs are much longer, we'll update our retrieval to return fewer title docs than chunked docs. We'll also update our retrieval to drop any chunks which are a part of one of the title docs:

In [212]:
from langchain_core.runnables import RunnableConfig


def content_search(input: dict, config: RunnableConfig) -> List[Document]:
    return vectorstore.similarity_search_with_score(
        input["query"].content_search,
        filter=input["filter"],
        k=config["configurable"]["content_k"],
    )


def title_search(input: dict, config: RunnableConfig) -> List[Document]:
    title_docs_scores = title_vectorstore.similarity_search_with_score(
        input["query"].title_search,
        filter=input["filter"],
        k=config["configurable"]["title_k"],
    )
    docs = docstore.mget(
        [title_doc.metadata["source"] for title_doc, _ in title_docs_scores]
    )
    scores = [scores for _, scores in title_docs_scores]
    return list(zip(docs, scores))


def dedup(input: dict):
    titles = [doc.metadata["source"] for doc, _ in input["title_docs"]]
    content_docs = []
    for doc, score in input["content_docs"]:
        if doc.metadata["source"] not in titles:
            content_docs.append((doc, score))
    input["content_docs"] = content_docs
    return input


search = (
    RunnablePassthrough.assign(content_docs=content_search, title_docs=title_search)
    | dedup
)

retrieval = query_analyzer_with_examples | queries_and_filters | search.map()

In [213]:
results = retrieval.invoke(
    "what's RAG", config={"configurable": {"content_k": 10, "title_k": 3}}
)
len(results[0]["content_docs"]), len(results[0]["title_docs"])

(7, 3)

Now we can set up the actual question-answering portion of our Q&A application. Again, for simplicity we'll just dump all the retrieved documents into a single prompt and let the model answer based off that:

In [227]:
from langchain_core.output_parsers import StrOutputParser


def format_docs(input: list):
    docs = []
    for query_results in input:
        docs.extend(query_results["content_docs"])
        docs.extend(query_results["title_docs"])
    return "\n\n".join(doc.page_content for doc, _ in docs)


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user question ONLY using the following up-to-date context. Do not worry about your last update date:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0)
parser = StrOutputParser()
qa_chain = (
    {"question": RunnablePassthrough(), "context": retrieval | format_docs}
    | prompt
    | llm
    | parser
)

In [228]:
for chunk in qa_chain.stream(
    "what are some updates the team has posted about LangGraph in 2024",
    config={"configurable": {"content_k": 10, "title_k": 3}},
):
    print(chunk, flush=True, end="")

In 2024, the LangGraph team has been actively working on enhancing the capabilities and applications of LangGraph, focusing on multi-agent workflows, collaboration, and hierarchical agent teams. Here are some key updates and developments:

1. **Multi-Agent Collaboration**: The team introduced concepts for multi-agent collaboration, where multiple agents work on the same state of messages, sharing information and tasks to achieve complex goals. This approach allows for a more collaborative and efficient problem-solving process, where agents can leverage each other's strengths.

2. **Agent Supervisor Model**: A new model called the "Agent Supervisor" was introduced, where a supervisor agent oversees the operation of multiple independent agents. Each agent works on its task and returns the final answer to the supervisor, which then decides the next course of action. This model allows for a clear separation of concerns and can enable more complex workflows by combining the outputs of vario

And here's what the trace would look like for this call: https://smith.langchain.com/public/9a70236f-5510-443c-a9da-084d9d07345d/r

## 🚧 Under construction 🚧 — Sorting: Going beyond search

In [81]:
question = "What has LangChain released lately?"
queries = [
    LangChainYouTubeSearch(sort_by="publish_date", relevance_rank=1),
]
examples.append({"input": question, "tool_calls": queries})

In [82]:
question = "What are the most popular videos about RAG?"
queries = [
    TutorialSearch(title_search="RAG", sort_by="view_count", relevance_rank=1),
    TutorialSearch(content_search="RAG", sort_by="view_count", relevance_rank=2),
    TutorialSearch(
        content_search="retrieval augmented generation",
        sort_by="view_count",
        relevance_rank=3,
    ),
    TutorialSearch(content_search="retrieval", sort_by="view_count", relevance_rank=4),
]
examples.append({"input": question, "tool_calls": queries})

In [86]:
question = "What are some short videos about agentic systems with local models?"
queries = [
    TutorialSearch(
        content_search="how to build an agent using a local model",
        sort_by="length",
        sort_order="ascending",
        relevance_rank=1,
    ),
    TutorialSearch(
        title_search="agent local model",
        sort_by="length",
        sort_order="ascending",
        relevance_rank=2,
    ),
]
examples.append({"input": question, "tool_calls": queries})

In [90]:
query_analyzer.invoke("Recent talks about local llms")

[LangChainYouTubeSearch(content='local LLMs', title=None, min_view_count=None, max_view_count=None, earliest_publish_date=None, latest_publish_date=None, min_length_sec=None, max_length_sec=None, sort_by='publish_date', sort_order='descending', relevance_rank=1)]

In [212]:
query_analyzer.invoke("new videos about streaming state machines")

[LangChainYouTubeSearch(content='new videos about streaming state machines', title=None, min_view_count=None, max_view_count=None, earliest_publish_date=None, latest_publish_date=None, min_length_sec=None, max_length_sec=None, sort_by='relevance', sort_order='descending', relevance_rank=1)]